In [ ]:
# Imports
import sys
sys.path.append("../../")
#import json
import urllib

#import utils
from utils_nlp.common.timer import Timer
from utils_nlp.azureml import azureml_utils

In [ ]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>"
)

In [ ]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')